online version of this notebook : https://colab.research.google.com/drive/12iz5_mTOmTa0oyn5IZZYnEskVUonViVW#scrollTo=VSTgsilO1_ur

In [1]:
import tensorflow as tf
import keras
from keras.optimizers import Adam
from keras.layers import Dense
from keras.models import Model, Sequential
from keras.activations import relu, softmax
from keras.losses import SparseCategoricalCrossentropy



In [2]:
def get_feature_extractor():
  raise NotImplementedError()

def get_internal_slicer():
  raise NotImplementedError()


def get_rx_decoder():
  raise NotImplementedError()

def phase_multiply(internally_sliced_y,h):
  raise NotImplementedError()

## PARAMS

In [3]:
SLICED_Y_LENGTH = 16
BATCH_SIZE =  1

# in teh feature extractor path "f" : design param
# Our experiments have shown that even a
# small number of features, e.g., F = 4, significantly improves
# the performance.
N_FEATURES_EXTRACTED = 8

In [4]:
def R2C(a):

    aa = tf.cast(tf.reshape(a,shape=(BATCH_SIZE,-1,2)),tf.float32)

    aaa = tf.complex(aa[:,:,0],aa[:,:,1])
    return aaa

def C2R(a):
    real, imag = tf.expand_dims(tf.math.real(a),axis=2) ,tf.expand_dims(tf.math.imag(a), axis=2)
    R = tf.concat((real,imag),axis=2)
    R = tf.reshape(R , (BATCH_SIZE,-1)  )
    return R


## Main Blocks in the Sequence Decoder

In [5]:

class FeatureExtractor(Model):
    def __init__(self):
        super().__init__()

        self.cf1 = Dense(256)

        self.cf2 = Dense(N_FEATURES_EXTRACTED)

        self.cf_state = Dense(8,name="featureExtractor_stateFC")

    def call(self,sliced_y,prev_state_FE):
        print("sliced_y.shape", sliced_y.shape)
        
        # combine the sliced_y and prev_state_FE
        sliced_y = tf.concat([sliced_y,prev_state_FE],axis=1)
        
        sliced_y = self.cf1(sliced_y)
        sliced_y = relu(sliced_y)
        
        state_FE = self.cf_state(sliced_y) # state calculated here
        
        sliced_y = self.cf2(state_FE)
        
        return sliced_y,state_FE

class PhaseEstimator(Model):
    def __init__(self):
        super().__init__()

        self.cf1 = Dense(256)
        self.cf2 = Dense(2)

        self.cf_state = Dense(8,name="PhaseEstimator_stateFC")


    def call(self,sliced_y,prev_state_PE):
        # combine sliced_y and prev_state_PE
        sliced_y = tf.concat([sliced_y,prev_state_PE],axis=1)
        sliced_y = self.cf1(sliced_y)
        sliced_y = relu(sliced_y)
        
        state_PE = self.cf_state(sliced_y) # state calculated here
        
        sliced_y = self.cf2(state_PE)
        
        return sliced_y,state_PE


class Rx_Decoder_old(Model):
    def __init__(self):
        super().__init__()

        self.cf1 = Dense(256)
        self.cf2 = Dense(256)
        self.cf3 = Dense(16)

    def call(self,concat):

        concat = self.cf1(concat)
        concat = relu(concat)
        concat = self.cf2(concat)
        concat = relu(concat)

        concat = self.cf3(concat)

        # do not use softmax here : put from logit  = True in loss func
        # concat = softmax(concat)

        return concat


class Rx_Decoder_new(Model):
    def __init__(self):
        super().__init__()

        self.cf1 = Dense(256)
        self.cf2 = Dense(256)
        self.cf3 = Dense(16,name="final_out_cf3")

        # useless
        #self.cf4_state = Dense(8,name="state_dense_cf4")

    def call(self,concat):

        concat = self.cf1(concat)
        concat = relu(concat)
        concat = self.cf2(concat)
        concat = relu(concat)

        # state = self.cf4_state(concat)
        concat = self.cf3(concat)



        # do not use softmax here : put from logit  = True in loss func
        # concat = softmax(concat)

        return concat

 
    
    
class InternalSlicer(Model):
    def __init__(self,l1,l2,complex_length):
        super().__init__()

        # define the slice boundaries
        mid = complex_length // 2
        self.start = mid - l1
        self.end = mid + l2 + 1

    def call(self,sliced_y):
       
        ret = C2R(R2C(sliced_y)[:, self.start:self.end])
       
        return ret


def phase_multiply(internally_sliced_y,estimated_phase):
    # (a,b) * (c,d) = (ac-bd,ad+bc)
   
    internally_sliced_y_complex = R2C(internally_sliced_y)
    estimated_phase_complex = R2C(estimated_phase)
    phase_corrected_complex = estimated_phase_complex * internally_sliced_y_complex

    phase_corrected = C2R(phase_corrected_complex)
    return phase_corrected






## Fake data syn

In [6]:
# generate fake data
m = 512* 2** 2
X = tf.random.normal(shape=(m,SLICED_Y_LENGTH),
                     mean=0,
                     stddev=1)

Y = tf.random.uniform(shape=(m,1),
                      minval=0,
                      maxval=16,
                      dtype=tf.int32)
# Y = keras.utils.to_categorical(Y,16)


## Main Model : Sequence Decoder Class

In [ ]:
# sequence decoder


class SequenceDecoder(Model):

    def __init__(self,take_prev_phase_state=False):
        super(SequenceDecoder,self).__init__()

        self.take_prev_phase_state = take_prev_phase_state

        self.feature_extractor = FeatureExtractor()
        self.phase_estimator = PhaseEstimator()
        self.internal_slicer = InternalSlicer(l1=3,l2=3,complex_length=SLICED_Y_LENGTH//2)
        
        if take_prev_phase_state:
            self.rx_decoder_RNN = Rx_Decoder_new()
        else:
            raise Exception("How here come??")
            #self.rx_decoder = Rx_Decoder_old()



    def call(self,sliced_y,prev_state_FE=None,prev_state_PE=None):

        if prev_state_PE is None:
            print(" How this none?")
            prev_state_PE = tf.constant(tf.zeros((X.shape[0],8)))
        if prev_state_FE is None:
            print(" How this none?")
            prev_state_FE = tf.constant(tf.zeros((X.shape[0],8)))


        # RNN conn starts here

        output_FE = self.feature_extractor(sliced_y,prev_state_FE=prev_state_FE)
        extracted_features,state_FE = output_FE[0], output_FE[1]
        


        output_PE = self.phase_estimator(sliced_y,prev_state_PE=prev_state_PE)
        estimated_phase,state_PE = output_PE[0], output_PE[1]

        # RNN conn ends here

        internally_sliced_y = self.internal_slicer(sliced_y)

       

        phase_corrected_ = phase_multiply(internally_sliced_y,estimated_phase)

        concat = tf.concat((extracted_features,phase_corrected_),axis=1)
        if self.take_prev_phase_state:
            st_hat = self.rx_decoder_RNN(concat)
            return (st_hat,state_FE,state_PE)
        else:
            raise Exception("How came here????")
            print("--PROBLEM--")
            st_hat = self.rx_decoder(concat)
            return st_hat



    def custom_train(self,X,Y,epochs=1): # X =  vertically stacked sliced_y, y = message index
        
        # tarin per each time step
        for _ in range(epochs):
            
            # temp_prev_state_PE = [tf.constant(tf.zeros((X.shape[0],8)))] #append the last PE state here
            # temp_prev_state_FE = [tf.constant(tf.zeros((X.shape[0],8)))] #append the last FE state here
            
            temp_prev_state_PE = tf.constant(tf.zeros((1,8)))
            temp_prev_state_FE = tf.constant(tf.zeros((1,8)))
            
            loss_acc = 0
            
            for i in range(X.shape[0]):
                print(f"iterration : {i}")
                x =  tf.expand_dims(X[i,:],axis=0)
             
                y = tf.expand_dims(Y[i,:],axis=0)
              
                with tf.GradientTape() as tape:
                    output = self.call(x,
                                       prev_state_PE=temp_prev_state_PE,
                                       prev_state_FE=temp_prev_state_FE)
                    
                    st_hat,state_FE,state_PE = output[0], output[1], output[2]
                    loss = self.compiled_loss(y,st_hat)

                    #temp_prev_state = state ###### assign add dala balanna
                   
                    temp_prev_state_FE = (state_FE)
                    temp_prev_state_PE = (state_PE)

                grads = tape.gradient(loss,self.trainable_variables)
                self.optimizer.apply_gradients(zip(grads,self.trainable_variables))

                loss_acc += loss.numpy() / X.shape[0] # take the mean
                print("loss (individual): ", loss.numpy())
            print(f'Epoch  : {_}/{epochs} --> Loss = {loss_acc}')
            
        # returning the final batch's loss
        return loss_acc




In [8]:
X.shape

TensorShape([2048, 16])

In [13]:
# test the SD

mySD =   SequenceDecoder(take_prev_phase_state=True)

mySD.compile(optimizer=Adam(learning_rate=1e-2),
             loss=SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])


mySD.custom_train(X,Y)





iterration : 0
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.7980328, shape=(), dtype=float32)
iterration : 1
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.729481, shape=(), dtype=float32)
iterration : 2
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(4.283499, shape=(), dtype=float32)
iterration : 3
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(0.050874222, shape=(), dtype=float32)
iterration : 4
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(5.9554043, shape=(), dtype=float32)
iterration : 5
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(3.2700474, shape=(), dtype=float32)
iterration : 6
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(13.070494, shape=(), dtype=float32)
iterration : 7
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(15.69781, shape=(), dtype=float32)
iterration : 8
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(7.821364, shape=(), dtype=float32)
iterration : 9
sliced_y.shape (1, 16)
loss (individual):  tf.Tenso

sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.0275347, shape=(), dtype=float32)
iterration : 89
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.9088423, shape=(), dtype=float32)
iterration : 90
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(3.034898, shape=(), dtype=float32)
iterration : 91
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(4.627492, shape=(), dtype=float32)
iterration : 92
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.1492634, shape=(), dtype=float32)
iterration : 93
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(3.323048, shape=(), dtype=float32)
iterration : 94
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(3.1000605, shape=(), dtype=float32)
iterration : 95
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.6817207, shape=(), dtype=float32)
iterration : 96
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(3.6887727, shape=(), dtype=float32)
iterration : 97
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.056

sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(1.9369575, shape=(), dtype=float32)
iterration : 167
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(7.96531, shape=(), dtype=float32)
iterration : 168
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(6.020131, shape=(), dtype=float32)
iterration : 169
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.8851194, shape=(), dtype=float32)
iterration : 170
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(3.3708456, shape=(), dtype=float32)
iterration : 171
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(3.2744634, shape=(), dtype=float32)
iterration : 172
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.7846441, shape=(), dtype=float32)
iterration : 173
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.6274576, shape=(), dtype=float32)
iterration : 174
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(4.8193607, shape=(), dtype=float32)
iterration : 175
sliced_y.shape (1, 16)
loss (individual):  tf.Ten

sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.6848097, shape=(), dtype=float32)
iterration : 246
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(3.0587082, shape=(), dtype=float32)
iterration : 247
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.6808367, shape=(), dtype=float32)
iterration : 248
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.873373, shape=(), dtype=float32)
iterration : 249
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.8705573, shape=(), dtype=float32)
iterration : 250
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.7400455, shape=(), dtype=float32)
iterration : 251
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.736325, shape=(), dtype=float32)
iterration : 252
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.7293465, shape=(), dtype=float32)
iterration : 253
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.7916505, shape=(), dtype=float32)
iterration : 254
sliced_y.shape (1, 16)
loss (individual):  tf.Te

sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.8666155, shape=(), dtype=float32)
iterration : 326
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.9868224, shape=(), dtype=float32)
iterration : 327
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(2.815435, shape=(), dtype=float32)
iterration : 328
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(3.0030336, shape=(), dtype=float32)
iterration : 329
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 330
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 331
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 332
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 333
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 334
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=flo

...

sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1709
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1710
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1711
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1712
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1713
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1714
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1715
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1716
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1717
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterrati

loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1800
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1801
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1802
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1803
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1804
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1805
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1806
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1807
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1808
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1809
sliced_y.shap

sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1883
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1884
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1885
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1886
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1887
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1888
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1889
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1890
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1891
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterrati

sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1976
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1977
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1978
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1979
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1980
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1981
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1982
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1983
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterration : 1984
sliced_y.shape (1, 16)
loss (individual):  tf.Tensor(nan, shape=(), dtype=float32)
iterrati

nan

In [10]:
# mySD.build((2048,16))